In [37]:
import math
import cv2
import numpy as np

focLenInPixels = 1047
baseDist = 2.2244
widthRes = 1920
heightRes = 1080
lineOpoToRCamAngle_xComp = 0.0
lineOpoToRCamAngle_yComp = 0.0
depthInInch = 0.0

def performTriangulation(leftXoffset, rightXoffset):
        global lineOpoToRCamAngle_xComp, lineOpoToRCamAngle_yComp, depthInInch
        # inside angle of the leftCam in triangulation
        leftCamAngle = 0.0
        angle1 = math.degrees(math.atan(abs(leftXoffset) / focLenInPixels))
        if leftXoffset > 0.0:
            leftCamAngle = 90.0 - angle1
        elif leftXoffset < 0.0:
            leftCamAngle = 90.0 + angle1
        else:  # leftXoffset is zero
            leftCamAngle = 90.0
        lCamAngle = leftCamAngle
        
        # inside angle of the rightCam in triangulation
        rightCamAngle = 0.0
        angle2 = math.degrees(math.atan(abs(rightXoffset) / focLenInPixels))
        if rightXoffset > 0.0:
            rightCamAngle = 90.0 + angle2
        elif rightXoffset < 0.0:
            rightCamAngle = 90.0 - angle2
        else:  # rightXoffset is zero
            rightCamAngle = 90.0
        rCamAngle = rightCamAngle
        cCamAngle = 180.0 - (lCamAngle + rCamAngle)

        """
        Assuming triangle with three corners having A,B and C angles and sides opposite to it respectively, a,b and c,
        we can use LAW OF SINES
            -> a/sin(A) = b/sin(B) = c/sin(C)
        - In our case we have angle near leftCam, angle near rightCam and the distance between two cameras - baseDist
        - Now, we can find length of any one side using 
        """
        try:
            knownRatio = baseDist / math.sin(math.radians(cCamAngle))
            lineOpoToRCamAngle = math.sin(math.radians(rightCamAngle)) * knownRatio
            
            depthInInch = lineOpoToRCamAngle * math.sin(math.radians(leftCamAngle))
            lineOpoToRCamAngle_yComp = depthInInch
            lineOpoToRCamAngle_xComp = lineOpoToRCamAngle * math.cos(math.radians(leftCamAngle))
            
        except Exception as e:
            depthInInch = 0.0
            print(f"error {e} encountered!")
            
        if type(depthInInch) == float and depthInInch < 0.0:
            print("Most likely your right and left camera is in wrong order!")

def drawImageWithTriangle() -> np.ndarray:
    squareLen = min([widthRes//3,heightRes//3])
    dimThreshold = 0.8 * squareLen   # traingle should be 80% of total image
    triangleShift = 0.1 * squareLen  # traingle has to move 10% in X direction
    
    # calculating triangle coordinates
    coOrd = []
    coOrd.append([0.0,0.0])             # left triangle point - camera1
    coOrd.append([baseDist,0.0])        # right triangle point - camera2
    coOrd.append([lineOpoToRCamAngle_xComp,lineOpoToRCamAngle_yComp])   # calculated cetner point
    coOrd = np.array([[i[0], i[1]] for i in coOrd], np.float32) # converting to np array
    
    # we already know the height of the triangle, now lets find max width as well
    triangleMaxWidth = np.max(coOrd[:,0]) - np.min(coOrd[:,0])
    if depthInInch > 0.0:
        triangleMaxHeight = depthInInch
        Scale = 1.0
        if triangleMaxWidth > triangleMaxHeight:
            Scale = dimThreshold/triangleMaxWidth
        else:
            Scale = dimThreshold/triangleMaxHeight
        
        # scale & shift traingle coordinates. Also, images have top left (0,0), so invert the triangle
        coOrd = coOrd*[Scale, Scale]
        coOrd = coOrd+[triangleShift,triangleShift]
        coOrd = [0,squareLen] - coOrd
        coOrd *= [-1.0,1.0]
        coOrd = coOrd.astype(np.int32)
        
        # draw this coordinates on a blank image - let's create blank image
        frame = np.zeros((squareLen, squareLen, 3), dtype=np.uint8)
        # Reshape vertices into shape required by cv2.polylines
        vertices = coOrd.reshape((-1, 1, 2))
        # Draw the triangle on the image
        frame = cv2.polylines(frame, [vertices], isClosed=True, color=(0, 0, 255), thickness=2)
        # adding text to the image to quit once done tuning
        frame = cv2.putText(
            img=frame,
            text=f"A",
            org=coOrd[0]-[25,0],
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=0.75,
            color=(225, 225, 255),
            thickness=1,
        )
        frame = cv2.putText(
            img=frame,
            text=f"B",
            org=coOrd[1]+[5,0],
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=0.75,
            color=(225, 225, 255),
            thickness=1,
        )
        frame = cv2.putText(
            img=frame,
            text=f"C",
            org=coOrd[2]+[-5,-5],
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=0.75,
            color=(225, 225, 255),
            thickness=1,
        )
        
        return frame
    else:
        return None

In [38]:
performTriangulation(-500, -350)
img = drawImageWithTriangle()
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()